In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
print(os.getcwd())
train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
print(train_data.shape)
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
print(test_data.shape)

/kaggle/working
(1460, 81)
(1459, 80)


In [3]:
# Always make a copy of the dataframe so you can use it again from starting.
import copy
train_data2 = copy.deepcopy(train_data)
test_data2 = copy.deepcopy(test_data)
print(train_data2.shape)
print(test_data2.shape)

(1460, 81)
(1459, 80)


In [4]:
## remove the missing columns > 20%
missing_percent = train_data.isna().mean().sort_values(ascending=False)*100
missing_percent.head(20)
columns_to_drop = list(missing_percent[missing_percent > 20].index)
train_data = train_data.drop(columns_to_drop, axis=1)
test_data = test_data.drop(columns_to_drop, axis=1)
print(train_data.shape)
print(test_data.shape)

(1460, 76)
(1459, 75)


In [5]:
# Now what we will try to do we will separate out the target variable and input variable so we should not make any
# changes in the target variable while performing the process on the input variable
y_train = train_data['SalePrice']
x_train = train_data.drop(['Id','SalePrice'],axis=1)
x_test = test_data.drop(['Id'],axis=1)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(1460, 74)
(1459, 74)
(1460,)


In [6]:
## Finding out the different type of column i.e. Categorical into nominal and ordinal and then the numerical
# For Nominal and ordinal we will take the cut off of 10
def identify_categorical(df):
    nominal_cols = []
    ordinal_cols = []
    numerical_cols = []
    for col in df.columns:
        if df[col].dtype == 'object':
            if df[col].nunique() <= 10:
                ordinal_cols.append(col)
            else:
                nominal_cols.append(col)
        elif df[col].dtype != 'object':
            numerical_cols.append(col)
    return nominal_cols, ordinal_cols, numerical_cols

In [7]:
train_data_nominal,train_data_ordinal,train_data_numerical= identify_categorical(x_train)
test_data_nominal,test_data_ordinal,test_data_numerical= identify_categorical(x_test)
print(len(train_data_nominal))
print(len(test_data_nominal))
print(len(train_data_ordinal))
print(len(test_data_ordinal))
print(len(train_data_numerical))
print(len(test_data_numerical))

3
3
35
35
36
36


In [8]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

def impute_encode(df, num_cols, ordinal_cols, cat_cols, ordinal_cats=None):
    """
    Imputes missing values in both numerical and categorical columns, and encodes them
    using ordinal encoding for ordinal categories and one-hot encoding for nominal categories.
    
    Parameters:
    df (pandas.DataFrame): The input DataFrame.
    num_cols (list): A list of column names that contain numerical data.
    ordinal_cols (list): A list of column names that contain ordinal categories.
    cat_cols (list): A list of column names that contain nominal categories.\
    
    Returns:
    pandas.DataFrame: The transformed DataFrame.
    """
    # Impute missing values in numerical columns
    num_imputer = SimpleImputer(strategy='mean')
    df[num_cols] = num_imputer.fit_transform(df[num_cols])
    
    # Impute missing values in categorical columns
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])
    
    # Impute missing values in cat_cols before one-hot encoding
    df[ordinal_cols] = cat_imputer.fit_transform(df[ordinal_cols])
    
    # Encode ordinal categories using ordinal encoding
    # ordinal_cats = ['low', 'medium', 'high']  # Example ordinal categories
    if ordinal_cats is not None:
        ordinal_encoder = OrdinalEncoder(categories=[ordinal_cats], dtype=int)
    else:
        ordinal_encoder = OrdinalEncoder(dtype=int)
    df[ordinal_cols] = ordinal_encoder.fit_transform(df[ordinal_cols])
    
    # Encode nominal categories using one-hot encoding
    onehot_encoder = OneHotEncoder()
    df = pd.concat([df.drop(cat_cols, axis=1),
                    pd.DataFrame(onehot_encoder.fit_transform(df[cat_cols]).toarray(),
                                 columns=onehot_encoder.get_feature_names(cat_cols))],
                   axis=1)
    return df



In [9]:
x_train_impute = impute_encode(x_train, num_cols=train_data_numerical, cat_cols=train_data_nominal, ordinal_cols=train_data_ordinal)
x_test_impute = impute_encode(x_test, num_cols=test_data_numerical, cat_cols=test_data_nominal, ordinal_cols=test_data_ordinal)
print(x_train_impute.shape)
print(x_test_impute.shape)

(1460, 127)
(1459, 124)


In [10]:
set1 = set(x_train_impute.columns)
set2 = set(x_test_impute.columns)
diff = set1.symmetric_difference(set2)
print(diff)

{'Exterior1st_Stone', 'Exterior2nd_Other', 'Exterior1st_ImStucc'}


In [11]:
# These onehot columns are missing in the test data, so drop them from the
# training data or we might overfit on them
x_train_impute.drop(diff, axis=1, inplace=True)
print(x_train_impute.shape)
print(x_test_impute.shape)

(1460, 124)
(1459, 124)


In [12]:
### Further we will perform the normal distribution for the numerical values by taking log(feature + 1).
from scipy.stats import skew
skewed = x_train_impute[train_data_numerical].apply(lambda x: skew(x.dropna().astype(float)))
print(skewed)
skewed = skewed[skewed > 0.75]
skewed = skewed.index
skewed

MSSubClass        1.406210
LotFrontage       2.382499
LotArea          12.195142
OverallQual       0.216721
OverallCond       0.692355
YearBuilt        -0.612831
YearRemodAdd     -0.503044
MasVnrArea        2.673661
BsmtFinSF1        1.683771
BsmtFinSF2        4.250888
BsmtUnfSF         0.919323
TotalBsmtSF       1.522688
1stFlrSF          1.375342
2ndFlrSF          0.812194
LowQualFinSF      9.002080
GrLivArea         1.365156
BsmtFullBath      0.595454
BsmtHalfBath      4.099186
FullBath          0.036524
HalfBath          0.675203
BedroomAbvGr      0.211572
KitchenAbvGr      4.483784
TotRmsAbvGrd      0.675646
Fireplaces        0.648898
GarageYrBlt      -0.667488
GarageCars       -0.342197
GarageArea        0.179796
WoodDeckSF        1.539792
OpenPorchSF       2.361912
EnclosedPorch     3.086696
3SsnPorch        10.293752
ScreenPorch       4.117977
PoolArea         14.813135
MiscVal          24.451640
MoSold            0.211835
YrSold            0.096170
dtype: float64


Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal'],
      dtype='object')

In [13]:
# Normalize the data and then see both the Kurtosis and Skewness of the data
x_train_impute[skewed] = np.log1p(x_train_impute[skewed])
x_test_impute[skewed] = np.log1p(x_test_impute[skewed])

In [14]:
y_train_norm = np.log1p(y_train)

In [15]:
skewed = x_train_impute[train_data_numerical].apply(lambda x: skew(x.dropna().astype(float)))
skewed = skewed[skewed > 0.75]
skewed = skewed.index
skewed

Index(['BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal'],
      dtype='object')

In [16]:
# kurtosis = train_data2[num_cols].apply(lambda x: kurtosis(x.dropna().astype(float)))
import scipy.stats as stats
import numpy as np
kurtosis_data2 = x_train_impute[train_data_numerical].apply(lambda x: stats.kurtosis(x.dropna().astype(float)))
kurtosis_data2 = kurtosis_data2[kurtosis_data2 > 10]
kurtosis_2 = kurtosis_data2.index
kurtosis_2

Index(['TotalBsmtSF', 'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr',
       '3SsnPorch', 'PoolArea', 'MiscVal'],
      dtype='object')

In [17]:
common1 = list(set(skewed.tolist()) & set(kurtosis_2.tolist()))
type(common1)

list

In [18]:
x_train_impute = x_train_impute.drop(common1, axis=1)
x_test_impute = x_test_impute.drop(common1, axis=1)
print(x_train_impute.shape)
print(x_test_impute.shape)

(1460, 118)
(1459, 118)


In [19]:
x_train_impute_array = x_train_impute.to_numpy()
x_test_impute_array = x_test_impute.to_numpy()

In [20]:
from sklearn.feature_selection import f_regression, SelectKBest
# configure to select all features
fs = SelectKBest(score_func=f_regression, k='all')
# learn relationship from training data
fs.fit(x_train_impute_array, y_train_norm)

SelectKBest(k='all', score_func=<function f_regression at 0x7f023c13dd40>)

In [21]:
x_train_impute_sel = x_train_impute.iloc[:,(i for i in range(len(fs.scores_)) if fs.scores_ [i] > 80)]
x_train_impute_sel.shape

(1460, 38)

In [22]:
x_test_impute_sel=x_test_impute[x_train_impute_sel.columns]

In [23]:
x_train_impute_sel_array = x_train_impute_sel.to_numpy()
x_test_impute_sel_array = x_test_impute_sel.to_numpy()

In [24]:
## Applying multiple regression model to see which is the best 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score

# Define a list of regression models to compare
models = [LinearRegression(),
          Ridge(alpha=1.0),
          Lasso(alpha=1.0),
          DecisionTreeRegressor(),
          RandomForestRegressor(n_estimators=100),
          GradientBoostingRegressor(n_estimators=100),
          AdaBoostRegressor(n_estimators=100)]

# Evaluate each model using cross-validation
for model in models:
    scores = cross_val_score(model, x_train_impute_sel_array, y_train_norm, cv=5,scoring='neg_root_mean_squared_error')
    print("{}: {:.2f} (+/- {:.2f})".format(model.__class__.__name__, -scores.mean(), scores.std()))

for model in models:
    scores = cross_val_score(model, x_train_impute_sel_array, y_train_norm, cv=5,scoring='r2')
    print("{}: {:.2f} (+/- {:.2f})".format(model.__class__.__name__, scores.mean(), scores.std()))


LinearRegression: 0.14 (+/- 0.02)
Ridge: 0.14 (+/- 0.02)
Lasso: 0.27 (+/- 0.01)
DecisionTreeRegressor: 0.21 (+/- 0.01)
RandomForestRegressor: 0.15 (+/- 0.01)
GradientBoostingRegressor: 0.14 (+/- 0.01)
AdaBoostRegressor: 0.18 (+/- 0.00)
LinearRegression: 0.87 (+/- 0.03)
Ridge: 0.87 (+/- 0.03)
Lasso: 0.55 (+/- 0.04)
DecisionTreeRegressor: 0.73 (+/- 0.03)
RandomForestRegressor: 0.86 (+/- 0.01)
GradientBoostingRegressor: 0.88 (+/- 0.01)
AdaBoostRegressor: 0.80 (+/- 0.02)


In [25]:
y_train_norm

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

In [26]:
from sklearn.linear_model import Lasso

# Fit the Lasso model to the data
lasso = Lasso(alpha=0.1) # alpha is the regularization parameter
lasso.fit(x_train_impute_array, y_train_norm)

# Get the coefficients and print the selected features
coefficients = lasso.coef_
selected_features = np.where(coefficients != 0)[0]
print("Selected features:", selected_features)


Selected features: [14 16 17 29 39 46 50 53]


In [27]:
x_train_impute_lasso_features = x_train_impute.iloc[:,selected_features]
x_train_impute_lasso_features_array = x_train_impute_lasso_features.to_numpy()

In [28]:
## Applying multiple regression model to see which is the best 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score

# Define a list of regression models to compare
models = [LinearRegression(),
          Ridge(alpha=1.0),
          Lasso(alpha=1.0),
          DecisionTreeRegressor(),
          RandomForestRegressor(n_estimators=100),
          GradientBoostingRegressor(n_estimators=100),
          AdaBoostRegressor(n_estimators=100)]

# Evaluate each model using cross-validation
for model in models:
    scores = cross_val_score(model, x_train_impute_lasso_features_array, y_train_norm, cv=5,scoring='neg_root_mean_squared_error')
    print("{}: {:.2f} (+/- {:.2f})".format(model.__class__.__name__, -scores.mean(), scores.std()))

for model in models:
    scores = cross_val_score(model, x_train_impute_lasso_features_array, y_train_norm, cv=5,scoring='r2')
    print("{}: {:.2f} (+/- {:.2f})".format(model.__class__.__name__, scores.mean(), scores.std()))


LinearRegression: 0.18 (+/- 0.01)
Ridge: 0.18 (+/- 0.01)
Lasso: 0.27 (+/- 0.01)
DecisionTreeRegressor: 0.24 (+/- 0.01)
RandomForestRegressor: 0.18 (+/- 0.00)
GradientBoostingRegressor: 0.17 (+/- 0.01)
AdaBoostRegressor: 0.20 (+/- 0.01)
LinearRegression: 0.80 (+/- 0.03)
Ridge: 0.80 (+/- 0.03)
Lasso: 0.55 (+/- 0.04)
DecisionTreeRegressor: 0.65 (+/- 0.04)
RandomForestRegressor: 0.80 (+/- 0.01)
GradientBoostingRegressor: 0.82 (+/- 0.01)
AdaBoostRegressor: 0.76 (+/- 0.03)


In [29]:
# Since Gradient Boost is the best out of all the method when we took the best features
# We will apply different Gradient boost method to see which one is the best
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor


# Define the list of models to train
models = [
    ("Gradient Boosting Machine (GBM)", GradientBoostingRegressor()),
    ("Extreme Gradient Boosting (XGBoost)", xgb.XGBRegressor()),
    ("Light Gradient Boosting Machine (LightGBM)", lgb.LGBMRegressor()),
    ("CatBoost", CatBoostRegressor(logging_level="Silent"))
]

# Train and evaluate each model
for name,model in models:
    scores = cross_val_score(model, x_train_impute_sel_array, y_train_norm, cv=5,scoring='neg_root_mean_squared_error')
    print("{}: {:.2f} (+/- {:.2f})".format(model.__class__.__name__, -scores.mean(), scores.std()))

for name,model in models:
    scores = cross_val_score(model, x_train_impute_sel_array, y_train_norm, cv=5,scoring='r2')
    print("{}: {:.2f} (+/- {:.2f})".format(model.__class__.__name__, scores.mean(), scores.std()))    


GradientBoostingRegressor: 0.14 (+/- 0.01)
XGBRegressor: 0.15 (+/- 0.01)
LGBMRegressor: 0.14 (+/- 0.01)
CatBoostRegressor: 0.14 (+/- 0.01)
GradientBoostingRegressor: 0.88 (+/- 0.01)
XGBRegressor: 0.86 (+/- 0.01)
LGBMRegressor: 0.87 (+/- 0.01)
CatBoostRegressor: 0.88 (+/- 0.01)


In [30]:
## The Gradient Boost has the best performance 
gbm = GradientBoostingRegressor()
gbm.fit(x_train_impute_sel_array,y_train_norm)

GradientBoostingRegressor()

In [31]:
y_predict=gbm.predict(x_test_impute_sel_array)

In [32]:
y_pred=np.exp(y_predict)

In [33]:
pred_df = pd.DataFrame(y_pred, index=test_data["Id"], columns=["SalePrice"])
pred_df.to_csv('submission.csv', header=True, index_label='Id')

In [34]:
os.getcwd()

'/kaggle/working'